In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# Both MNIST and Fashion-MNIST can be loaded from Keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

from tqdm import tqdm
from tqdm.autonotebook import tqdm, trange
from loop import TrainingLoop

In [2]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert image pixels to floats between 0 and 1
X_train = x_train / 255
X_test = x_test / 255

# Convert output to one hot encoding
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [3]:
from sklearn.model_selection import train_test_split
#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [4]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(layers.Input(shape = (28, 28, 1,)))
model.add(layers.Conv2D(32, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D( pool_size = (2, 2)))
model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation = "softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print( model.summary() )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
____________________________________________________

In [8]:
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

#validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
#validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True)

In [9]:
train_data = list(train_dataset)

In [10]:
train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

In [11]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [12]:
@tf.function
def validation_step(x_val, y_val):
    val_logits = model(x_val, training=False)
    val_acc_metric.update_state(y_val, val_logits)

In [13]:
epochs = 0

for epoch in range(epochs):    
    steps = trange(len(train_data), bar_format="{desc}\t{percentage:3.0f}% {r_bar}")
    for i in steps:
        step = i
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        
        loss_value = train_step(x_batch_train, y_batch_train)
        
        steps.set_description("Epoch " + str(epoch+1) + '/' + str(epochs) + "\tLoss: " + str(float(loss_value))[:6]
                              + "\tAccuracy: " + str(float(train_acc_metric.result()))[:6])
        
        if i == len(train_data)-1:
          
            for x_batch_val, y_batch_val in validation_dataset:
                validation_step(x_batch_val, y_batch_val)

            steps.set_description(steps.desc + "\tValidation accuracy: " + str(float(val_acc_metric.result()))[:6])

    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

## Using our custom class to train the model.

In [14]:
training = TrainingLoop(model, X_train, Y_train, loss_fn, optimizer, 
                        train_metrics=train_acc_metric, val_metrics=val_acc_metric, 
                        validation_split=0.1, batch_size=batch_size)

In [15]:
training.train(20)

Epoch 1/20	Loss: 1.6529	Accuracy: 0.7253: 	Validation accuracy: 0.7797: 	100% | 843/843 [00:03<00:00, 246.12it/s]
Epoch 2/20	Loss: 1.5955	Accuracy: 0.8253: 	Validation accuracy: 0.8576: 	100% | 843/843 [00:02<00:00, 398.85it/s]
Epoch 3/20	Loss: 1.5948	Accuracy: 0.8592: 	Validation accuracy: 0.8639: 	100% | 843/843 [00:02<00:00, 394.00it/s]
Epoch 4/20	Loss: 1.5843	Accuracy: 0.8694: 	Validation accuracy: 0.8681: 	100% | 843/843 [00:02<00:00, 401.20it/s]
Epoch 5/20	Loss: 1.5571	Accuracy: 0.8773: 	Validation accuracy: 0.8760: 	100% | 843/843 [00:02<00:00, 400.13it/s]
Epoch 6/20	Loss: 1.5678	Accuracy: 0.8833: 	Validation accuracy: 0.8805: 	100% | 843/843 [00:02<00:00, 400.15it/s]
Epoch 7/20	Loss: 1.5528	Accuracy: 0.8886: 	Validation accuracy: 0.8837: 	100% | 843/843 [00:02<00:00, 402.61it/s]
Epoch 8/20	Loss: 1.5447	Accuracy: 0.8905: 	Validation accuracy: 0.8832: 	100% | 843/843 [00:02<00:00, 396.63it/s]
Epoch 9/20	Loss: 1.5446	Accuracy: 0.8956: 	Validation accuracy: 0.8855: 	100% | 843/843 